In [2]:
from sympy import *
import numpy as np 
init_printing()

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D21, D22, D23 ,D31, D32, D33 = symbols('D11, D12, D13, D21, D22, D23 ,D31, D32, D33')
Exx, Ezz, Gxz = symbols('Exx, Ezz, Gxz')
stress_labels = ['Txx', 'Tzz', 'Txz']

#D11 = 2*eta; D12 = 0*eta; D13 = 0*eta;
#D21 = 0*eta; D22 = 2*eta; D23 = 0*eta;
#D31 = 0*eta; D32 = 0*eta; D33 = 1*eta;

Dv  = Matrix( [ [D11, D12, D13], [D21, D22, D23], [D31, D32, D33]  ] )
div = Exx+Ezz
Ed  = Matrix( [ [Exx-1/3*div], [Ezz-1/3*div], [Gxz] ] )
T   = Dv*Ed;
for i in range(len(stress_labels)):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )


Txx = D11*(0.66666666666666674*Exx - 0.33333333333333331*Ezz) + D12*(-0.33333333333333331*Exx + 0.66666666666666674*Ezz) + D13*Gxz;
Tzz = D21*(0.66666666666666674*Exx - 0.33333333333333331*Ezz) + D22*(-0.33333333333333331*Exx + 0.66666666666666674*Ezz) + D23*Gxz;
Txz = D31*(0.66666666666666674*Exx - 0.33333333333333331*Ezz) + D32*(-0.33333333333333331*Exx + 0.66666666666666674*Ezz) + D33*Gxz;


In [3]:
# Compressibility switch
comp = symbols('comp')

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')

inW,inE,inSu,inSv,inNu,inNv = symbols('inW,inE,inSu,inSv,inNu,inNv')

dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])
#----------------------------------------#
divW = 1/dx*(uC-uW) + 1/dz*(vNW-vSW)
ExxW = 1/dx*(uC-uW) - comp*1/3*divW;
EzzW = 0;
GxzW = 0
EdW  = Matrix([[inW*ExxW], [inW*EzzW], [inW*GxzW]])
TW   = Dv*EdW
TxxW = TW[0].subs(D11,'D11W')
#TxxW = 2* etaW* ExxW
#----------------------------------------#
divE = 1/dx*(uE-uC) + 1/dz*(vNE-vSE)
ExxE = 1/dx*(uE-uC) - comp*1/3*divE
EzzE = 0;
GxzE = 0
EdE  = Matrix([[inE*ExxE], [inE*EzzE], [inE*GxzE]])
TE   = Dv*EdE
TxxE = TE[0].subs(D11,'D11E')
#----------------------------------------#
ExxS = 0
EzzS = 0
GxzS = inSu*1/dz*(uC-uS) + inSv*1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TxzS = TS[2].subs(D33,'D33S')
#----------------------------------------#
ExxN = 0
EzzN = 0
GxzN = inNu*1/dz*(uN-uC) + inNv*1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TxzN = TN[2].subs(D33,'D33N')
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS)  
Fx = -Fx

for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cUc) + ';')

uW = D11W*inW*((1.0/3.0)*comp/dx - 1/dx)/dx;
uC = -(-D33N*inNu/dz - D33S*inSu/dz)/dz - (D11E*inE*((1.0/3.0)*comp/dx - 1/dx) - D11W*inW*(-1.0/3.0*comp/dx + 1.0/dx))/dx;
uE = -D11E*inE*(-1.0/3.0*comp/dx + 1.0/dx)/dx;
uS = -D33S*inSu/pow(dz, 2);
uN = -D33N*inNu/pow(dz, 2);
vSW = (1.0/3.0)*D11W*comp*inW/(dx*dz) - D33S*inSv/(dx*dz);
vSE = -1.0/3.0*D11E*comp*inE/(dx*dz) + D33S*inSv/(dx*dz);
vNW = -1.0/3.0*D11W*comp*inW/(dx*dz) + D33N*inNv/(dx*dz);
vNE = (1.0/3.0)*D11E*comp*inE/(dx*dz) - D33N*inNv/(dx*dz);


In [4]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')

inS,inN,inWu,inWv,inEu,inEv = symbols('inS,inN,inWu,inWv,inEu,inEv')

dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])
#----------------------------------------#
divS = 1/dz*(vC-vS) + 1/dx*(uSE-uSW)
ExxS = 0
EzzS = 1/dz*(vC-vS) - comp*1/3*divS;
GxzS = 0
EdS  = Matrix([[inS*ExxS], [inS*EzzS], [inS*GxzS]])
TS   = Dv*EdS
TzzS = TS[1].subs(D22,'D22S')
#----------------------------------------#
divN = 1/dz*(vN-vC) + 1/dx*(uNE-uNW)
ExxN = 0
EzzN = 1/dz*(vN-vC)  - comp*1/3*divN;
GxzN = 0
EdN  = Matrix([[inN*ExxN], [inN*EzzN], [inN*GxzN]])
TN   = Dv*EdN
TzzN = TN[1].subs(D22,'D22N')
#----------------------------------------#
ExxW = 0
EzzW = 0
GxzW = inWu*1/dz*(uNW-uSW) + inWv*1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxzW = TW[2].subs(D33,'D33W')
# #----------------------------------------#
ExxE = 0
EzzE = 0
GxzE = inEu*1/dz*(uNE-uSE) + inEv*1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxzE = TE[2].subs(D33,'D33E')
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
Fz = -Fz
for i in range(9):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cVc) + ';') #print(cUc)

vW = -D33W*inWv/pow(dx, 2);
vC = -(D22N*inN*((1.0/3.0)*comp/dz - 1/dz) - D22S*inS*(-1.0/3.0*comp/dz + 1.0/dz))/dz - (-D33E*inEv/dx - D33W*inWv/dx)/dx;
vE = -D33E*inEv/pow(dx, 2);
vS = D22S*inS*((1.0/3.0)*comp/dz - 1/dz)/dz;
vN = -D22N*inN*(-1.0/3.0*comp/dz + 1.0/dz)/dz;
uSW = (1.0/3.0)*D22S*comp*inS/(dx*dz) - D33W*inWu/(dx*dz);
uSE = -1.0/3.0*D22S*comp*inS/(dx*dz) + D33E*inEu/(dx*dz);
uNW = -1.0/3.0*D22N*comp*inN/(dx*dz) + D33W*inWu/(dx*dz);
uNE = (1.0/3.0)*D22N*comp*inN/(dx*dz) - D33E*inEu/(dx*dz);
